In [2]:
_ = ! pip install faster-whisper

In [26]:
from faster_whisper import WhisperModel
model_size = "large-v3"
model = WhisperModel(model_size, device="cpu", compute_type="int8")

In [27]:
segments, info = model.transcribe("audio2.mp4", beam_size=5)

In [28]:
print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

Detected language 'zh' with probability 0.987605


In [29]:
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

[0.00s -> 2.00s] 翻译&字幕志愿者 杨茜茜. 父亲时钟
[2.00s -> 4.00s] 作词 杨茜茜. 父亲时钟
[4.00s -> 6.00s] 作曲 杨茜茜. 父亲时钟
[12.00s -> 14.00s] 哈喽 大家好
[14.00s -> 16.00s] 谢谢TED
[16.00s -> 18.00s] 谢谢父亲时钟的邀请
[18.00s -> 20.00s] 我是曾志晓女士
[22.00s -> 24.00s] 好正式
[24.00s -> 26.00s] 很可爱的主持人
[26.00s -> 28.00s] 好 我非常荣幸
[28.00s -> 30.00s] 可以跟大家分享
[30.00s -> 32.00s] 我最近最有感的四个字
[32.00s -> 34.00s] 突破框架
[34.00s -> 36.00s] 首先我想要先跟大家聊聊
[36.00s -> 38.00s] 确认一下大家对
[38.00s -> 40.00s] 突破框架四个字
[40.00s -> 42.00s] 是正面感受吗
[42.00s -> 44.00s] 可以 有些人说是
[44.00s -> 46.00s] 谢谢你理我 谢谢
[46.00s -> 48.00s] 是是是
[48.00s -> 50.00s] 好 那我们继续聊下去
[50.00s -> 52.00s] 如果是正面感受的话
[52.00s -> 54.00s] 我也是这样认同的
[54.00s -> 56.00s] 我认为突破框架不是一个
[56.00s -> 58.00s] 举动 或是一个做法
[58.00s -> 60.00s] 我觉得更重要它是一个
[60.00s -> 62.00s] 态度
[62.00s -> 64.00s] 一种精神
[64.00s -> 66.00s] 突破框架
[66.00s -> 68.00s] 我是一个很喜欢拆解
[68.00s -> 70.00s] 字面的人
[70.00s -> 72.00s] 我想要先跟大家分享
[72.00s -> 74.00s] 如果你认为
[74.00s -> 76.00s] 我是一个突破框架的
[76.00s -> 78.00s] 人
[78.00s -> 80.00s] 或是你认为我是一个
[80.00s -> 82.00

KeyboardInterrupt: 

In [21]:
for segment in segments:
    print(segment.text)

In [39]:
_ = ! pip install fastapi
_ = ! pip install uvicorn
_ = ! pip install python-multipart

In [36]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse
import uvicorn
import os
import uuid
import torch
from faster_whisper import WhisperModel

In [37]:
# Initialize FastAPI app
app = FastAPI(
    title="Faster Whisper Transcription API",
    description="Upload an audio file and receive its transcription using faster-whisper.",
    version="1.0.0"
)

# Load the Whisper model on startup
MODEL_SIZE = "tiny"  # other sizes: tiny, base, medium, large
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
whisper_model = WhisperModel(
    model_size_or_path=MODEL_SIZE,
    device=DEVICE,
    compute_type="int8"  # quantized inference
)


config.json:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

In [41]:
@app.post("/transcribe")
async def transcribe_audio(file: UploadFile = File(...)):
    # Validate file type
    if not file.content_type.startswith(("audio/", "video/")):
        raise HTTPException(status_code=400, detail="Unsupported file type")

    # Save upload to temporary file
    ext = file.filename.split('.')[-1]
    temp_path = f"/tmp/{uuid.uuid4()}.{ext}"
    contents = await file.read()
    with open(temp_path, "wb") as f:
        f.write(contents)

    try:
        # Perform transcription
        segments, info = whisper_model.transcribe(
            temp_path,
            beam_size=5  # adjust for quality vs speed
        )
        # Concatenate segment texts
        transcript = "".join([segment.text for segment in segments]).strip()
        result = {
            "text": transcript,
            "language": info.language,
            "language_probability": info.language_probability
        }
        return JSONResponse(content=result)
    finally:
        # Clean up temp file
        if os.path.exists(temp_path):
            os.remove(temp_path)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


AttributeError: 'FastAPI' object has no attribute 'run'